In [1]:
import pandas as pd
import requests
import bs4
import re
import os

# 1. 수집해야할 항목의 링크주소 수집

In [2]:
mainUrl = "https://www.calspia.go.kr/io/openapi/gid/selectOpenApiSvcListView.do"

In [3]:
mainresp = requests.get(mainUrl)

mainhtml = mainresp.text

In [4]:
mainbs = bs4.BeautifulSoup(mainhtml,"html.parser")

In [5]:
tableTag = mainbs.find(name="table", attrs={"id":"cmTable"})

In [6]:
inputTags = tableTag.findAll(name="input")

In [7]:
cmList = []

In [8]:
for i in range(0, len(inputTags)):
    cmValue = inputTags[i].attrs["value"]
    cmList.append(cmValue)

### 2. 링크주소 내 타이틀정보 및 컬럼세부정보 수집

In [9]:
preFixUrl = "https://www.calspia.go.kr/io/openapi/gid/selectOpenApiSvcDtlView.do?apiSvcCd="

In [10]:
for cm in range(0, len(cmList)):

    url = preFixUrl+cmList[cm]

    resp = requests.get(url)

    html = resp.text

    bs = bs4.BeautifulSoup(html,"html.parser")

    metadata = pd.read_excel("../input/datalake_meta22.xlsx")

    ### 기본정보 설정 및 파라미터 설정 ###
    SITENAME = "건설사업정보시스템"
    DATANAME = bs.find(name="h2", attrs={"class":"subnav_tit_lv1"}).text
    print(DATANAME)
    APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

    targetData = metadata.loc[metadata.자료명==DATANAME]
    # URL = targetData["URL"].values[0]
    # SERVICENAME = targetData["서비스키"].values[0]
    # SERVICENAME = SERVICENAME.split(".")[0]
    # REQPARAM = targetData["요청변수"].values[0]
    # REQPARAM = REQPARAM.split(",")
    # PRIMARYKEY = targetData["기본키"].values[0]

    test = bs.find(name="table", attrs={"class":"common_tbl", 
                                        "id":"prnItemTable"} )

    tbodyTag = bs.find(name="tbody", attrs={"id":"prnItemTableBody"} )

    columnList = []
    rowList = []

    tableRows = tbodyTag.findAll(name="tr")

    tableRowsLen = len(tableRows)

    dataString1 = DATANAME.split(" ")[0]
    dataString2 = DATANAME.split(" ")[1]
    if cmList[cm]== "CM_0000331":
        dataString1="공사현황"
    if cmList[cm]=="FM_0000332":
        dataString1="시설물"
    if (cm >=25) & (cm <=37):
        dataString1="보상"
    if cmList[cm]=="LC_0000145":
        dataString1="환매권"
    if cmList[cm]=="LC_0000146":
        dataString1="부동산소유사실"
    if cmList[cm]=="PT_0000190":
        dataString1="검토조직"
    if cmList[cm]=="PT_0000321":
        dataString1="바디"
    skipAct=False
    for row in range(0, tableRowsLen):

        searchString = re.sub(r"[\n\t\s\r]*", "", tableRows[row].text)

        if  (searchString.count(dataString1) >=1)|(skipAct)|(searchString.count(dataString2) >=1):
            skipAct=True
        else:
            continue

        eachRow = tableRows[row]

        columns = eachRow.findAll(name="td")

        for i in range(0, len(columns)):

            columnValue = columns[i].text
            columnValue = re.sub(r"[\n\t\s\r]*", "", columnValue)
            columnList.append(columnValue)

        rowList.append(columnList)

        columnList = []

    columnNames = ["컬럼명","컬럼타입","컬럼정보"]

    tableDefine = pd.DataFrame( rowList[:],columns = columnNames)

    tableList.append(tableDefine)

공사정보 목록


NameError: name 'tableList' is not defined

### 3. 수집된정보 엑셀 내 시트추가하여 저장

In [ ]:
for i in range(0, len(tableList)):
    # print(cmList[i])
    sheetName = "sheet"
    try: 
        sheetName = tableList[i].loc[0].컬럼정보
    except:
        sheetName = "sheet"
    sheetName = sheetName.replace("/","_")
    print(sheetName)
    df = tableList[i].loc[1:]
    ## 파일이 존재하지 않으면 생성
    ## 파일이 존재하지 않으면 생성
    try:
        if not os.path.exists("output.xlsx"):
            with pd.ExcelWriter("output.xlsx", mode="w", engine="openpyxl") as writer:
                df.to_excel(writer, index=False,sheet_name = sheetName)

        else:
            with pd.ExcelWriter("output.xlsx", mode="a", engine="openpyxl",if_sheet_exists="new") as writer:
                df.to_excel(writer, index=False,sheet_name = sheetName)  
    except Exception as e:
        print(e,i)